In [3]:
import os
import polars as pl

# Define the data folder path
data_folder = "data"

# Get a list of all Parquet files in the data folder that do not start with 'mavir'
parquet_files = [os.path.join(data_folder, f) for f in os.listdir(data_folder) if f.endswith(".parquet") and not f.startswith("mavir")]

# Initialize a variable to hold the merged DataFrame
merged_df = None

# Iterate through the Parquet files and merge them sequentially
for file in parquet_files:
    df = pl.read_parquet(file)
    if merged_df is None:
        merged_df = df
    else:
        merged_df = merged_df.join(df, on="Időpont", how="inner")

# The final merged DataFrame is stored in merged_df
print(merged_df)

shape: (210_588, 41)
┌────────────┬─────────┬──────────┬──────────┬───┬────────────┬────────────┬───────────┬───────────┐
│ Időpont    ┆ HU-UK   ┆ HU-SK    ┆ HU-SI    ┆ … ┆ Szélerőműv ┆ Szélerőműv ┆ Szélerőmű ┆ Szélerőmű │
│ ---        ┆ ---     ┆ ---      ┆ ---      ┆   ┆ ek tény -  ┆ ek tény -  ┆ vek tény  ┆ vek tény  │
│ datetime[μ ┆ f64     ┆ f64      ┆ f64      ┆   ┆ nettó      ┆ nettó      ┆ - bruttó  ┆ - bruttó  │
│ s, UTC]    ┆         ┆          ┆          ┆   ┆ kere…      ┆ üzem…      ┆ üze…      ┆ üze…      │
│            ┆         ┆          ┆          ┆   ┆ ---        ┆ ---        ┆ ---       ┆ ---       │
│            ┆         ┆          ┆          ┆   ┆ f64        ┆ f64        ┆ f64       ┆ f64       │
╞════════════╪═════════╪══════════╪══════════╪═══╪════════════╪════════════╪═══════════╪═══════════╡
│ 2018-12-31 ┆ 672.907 ┆ 1628.383 ┆ null     ┆ … ┆ 22.151     ┆ 21.346     ┆ null      ┆ null      │
│ 23:15:00   ┆         ┆          ┆          ┆   ┆            ┆       

In [ ]:
import os
import polars as pl

# Define the data folder path
data_folder = "data"

# Get a list of all Parquet files in the data folder
parquet_files = [os.path.join(data_folder, f) for f in os.listdir(data_folder) if f.endswith(".parquet")]

# Initialize a variable to hold the merged DataFrame
df_balance = None

# Iterate through the Parquet files
for file in parquet_files:
    df = pl.read_parquet(file)

    # If the file starts with 'mavir', create the 'Időpont' column
    if os.path.basename(file).startswith("mavir"):
        df = df.with_columns(
            (pl.col("Dátum").cast(pl.Utf8) + " " + pl.col("Kiegyenlítő-energia-elszámolási időszak").cast(pl.Utf8))
            .str.strptime(pl.Datetime, "%Y-%m-%d %H:%M:%S", strict=False)
            .dt.convert_time_zone("UTC")  # Ensure the datetime is timezone-aware
            .alias("Időpont")
        )
    else:
        # Ensure the 'Időpont' column is timezone-aware for non-'mavir' files
        df = df.with_columns(pl.col("Időpont").dt.convert_time_zone("UTC"))

    # Merge the DataFrame based on the 'Időpont' column
    if df_balance is None:
        df_balance = df
    else:
        df_balance = df_balance.join(df, on="Időpont", how="inner")

# The final merged DataFrame is stored in df_balance
print(df_balance)

shape: (210_633, 65)
┌────────────┬─────────┬──────────┬──────────┬───┬────────────┬────────────┬───────────┬───────────┐
│ Időpont    ┆ HU-UK   ┆ HU-SK    ┆ HU-SI    ┆ … ┆ Szélerőműv ┆ Szélerőműv ┆ Szélerőmű ┆ Szélerőmű │
│ ---        ┆ ---     ┆ ---      ┆ ---      ┆   ┆ ek tény -  ┆ ek tény -  ┆ vek tény  ┆ vek tény  │
│ datetime[μ ┆ f64     ┆ f64      ┆ f64      ┆   ┆ nettó      ┆ nettó      ┆ - bruttó  ┆ - bruttó  │
│ s, UTC]    ┆         ┆          ┆          ┆   ┆ kere…      ┆ üzem…      ┆ üze…      ┆ üze…      │
│            ┆         ┆          ┆          ┆   ┆ ---        ┆ ---        ┆ ---       ┆ ---       │
│            ┆         ┆          ┆          ┆   ┆ f64        ┆ f64        ┆ f64       ┆ f64       │
╞════════════╪═════════╪══════════╪══════════╪═══╪════════════╪════════════╪═══════════╪═══════════╡
│ 2019-01-01 ┆ 675.547 ┆ 1877.437 ┆ null     ┆ … ┆ 5.968      ┆ 5.116      ┆ null      ┆ null      │
│ 00:00:00   ┆         ┆          ┆          ┆   ┆            ┆       

In [8]:
# Ensure both DataFrames have the same type for the 'Időpont' column
df_balance = df_balance.with_columns(pl.col("Időpont").dt.replace_time_zone(None))
merged_df = merged_df.with_columns(pl.col("Időpont").dt.replace_time_zone(None))

# Join df_balance and merged_df on the 'Időpont' column
final_df = df_balance.join(merged_df, on="Időpont", how="inner")

# The resulting DataFrame is stored in final_df
print(final_df)

shape: (215_593, 105)
┌────────────┬─────────┬──────────┬──────────┬───┬────────────┬────────────┬───────────┬───────────┐
│ Időpont    ┆ HU-UK   ┆ HU-SK    ┆ HU-SI    ┆ … ┆ Szélerőműv ┆ Szélerőműv ┆ Szélerőmű ┆ Szélerőmű │
│ ---        ┆ ---     ┆ ---      ┆ ---      ┆   ┆ ek tény -  ┆ ek tény -  ┆ vek tény  ┆ vek tény  │
│ datetime[μ ┆ f64     ┆ f64      ┆ f64      ┆   ┆ nettó      ┆ nettó      ┆ - bruttó  ┆ - bruttó  │
│ s]         ┆         ┆          ┆          ┆   ┆ kere…      ┆ üzem…      ┆ üze…      ┆ üze…      │
│            ┆         ┆          ┆          ┆   ┆ ---        ┆ ---        ┆ ---       ┆ ---       │
│            ┆         ┆          ┆          ┆   ┆ f64        ┆ f64        ┆ f64       ┆ f64       │
╞════════════╪═════════╪══════════╪══════════╪═══╪════════════╪════════════╪═══════════╪═══════════╡
│ 2019-01-01 ┆ 675.547 ┆ 1877.437 ┆ null     ┆ … ┆ 5.968      ┆ 5.116      ┆ null      ┆ null      │
│ 00:00:00   ┆         ┆          ┆          ┆   ┆            ┆      

In [9]:
final_df = final_df.drop(["Dátum_right", "Kiegyenlítő-energia-elszámolási időszak_right"])
print(final_df)

shape: (215_593, 103)
┌────────────┬─────────┬──────────┬──────────┬───┬────────────┬────────────┬───────────┬───────────┐
│ Időpont    ┆ HU-UK   ┆ HU-SK    ┆ HU-SI    ┆ … ┆ Szélerőműv ┆ Szélerőműv ┆ Szélerőmű ┆ Szélerőmű │
│ ---        ┆ ---     ┆ ---      ┆ ---      ┆   ┆ ek tény -  ┆ ek tény -  ┆ vek tény  ┆ vek tény  │
│ datetime[μ ┆ f64     ┆ f64      ┆ f64      ┆   ┆ nettó      ┆ nettó      ┆ - bruttó  ┆ - bruttó  │
│ s]         ┆         ┆          ┆          ┆   ┆ kere…      ┆ üzem…      ┆ üze…      ┆ üze…      │
│            ┆         ┆          ┆          ┆   ┆ ---        ┆ ---        ┆ ---       ┆ ---       │
│            ┆         ┆          ┆          ┆   ┆ f64        ┆ f64        ┆ f64       ┆ f64       │
╞════════════╪═════════╪══════════╪══════════╪═══╪════════════╪════════════╪═══════════╪═══════════╡
│ 2019-01-01 ┆ 675.547 ┆ 1877.437 ┆ null     ┆ … ┆ 5.968      ┆ 5.116      ┆ null      ┆ null      │
│ 00:00:00   ┆         ┆          ┆          ┆   ┆            ┆      

In [ ]:
import polars as pl
from datetime import datetime, timezone

mavir_neg = pl.read_parquet("data/mavir_Neg_data.parquet").with_columns(
    pl.col("Dátum")
    .dt.combine(pl.col("Kiegyenlítő-energia-elszámolási időszak"))
    .dt.convert_time_zone(time_zone="Europe/Budapest")
    .alias("Időpont")
)
mavir_poz = pl.read_parquet("data/mavir_Poz_data.parquet").with_columns(
    pl.col("Dátum")
    .dt.combine(pl.col("Kiegyenlítő-energia-elszámolási időszak"))
    .dt.convert_time_zone(time_zone="Europe/Budapest")
    .dt.convert_time_zone("UTC")
    .alias("Időpont")
)
PV = pl.read_parquet("data/PV.parquet")
hatar_aramlas = pl.read_parquet("data/hatar_aramlas.parquet")
real_time_aggregated = pl.read_parquet("data/real_time_aggregated.parquet")
rendszerterheles = pl.read_parquet("data/rendszerterheles.parquet")
wind = pl.read_parquet("data/wind.parquet")

print(mavir_neg.shape)
print(mavir_poz.shape)
print(PV.shape)
print(hatar_aramlas.shape)
print(real_time_aggregated.shape)
print(rendszerterheles.shape)
print(wind.shape)
print()

print(mavir_neg["Időpont"].min())
print(mavir_poz["Időpont"].min())
print(PV["Időpont"].min())
print(hatar_aramlas["Időpont"].min())
print(real_time_aggregated["Időpont"].min())
print(rendszerterheles["Időpont"].min())
print(wind["Időpont"].min())
print()

print(mavir_neg["Időpont"].max())
print(mavir_poz["Időpont"].max())
print(PV["Időpont"].max())
print(hatar_aramlas["Időpont"].max())
print(real_time_aggregated["Időpont"].max())
print(rendszerterheles["Időpont"].max())
print(wind["Időpont"].max())
print()

print(f"mavir_neg Időpont type: {type(mavir_neg['Időpont'][0])}")
print(f"mavir_poz Időpont type: {type(mavir_poz['Időpont'][0])}")
print(f"PV Időpont type: {type(PV['Időpont'][0])}")
print(f"hatar_aramlas Időpont type: {type(hatar_aramlas['Időpont'][0])}")
print(f"real_time_aggregated Időpont type: {type(real_time_aggregated['Időpont'][0])}")
print(f"rendszerterheles Időpont type: {type(rendszerterheles['Időpont'][0])}")
print(f"wind Időpont type: {type(wind['Időpont'][0])}")
print()

print("mavir_neg Időpont sample:")
print(mavir_neg.select("Időpont").head())
print("mavir_poz Időpont sample:")
print(mavir_poz.select("Időpont").head())
print("PV Időpont sample:")
print(PV.select("Időpont").head())
print("hatar_aramlas Időpont sample:")
print(hatar_aramlas.select("Időpont").head())
print("real_time_aggregated Időpont sample:")
print(real_time_aggregated.select("Időpont").head())
print("rendszerterheles Időpont sample:")
print(rendszerterheles.select("Időpont").head())
print("wind Időpont sample:")
print(wind.select("Időpont").head())

df = pl.concat(
    [
        mavir_neg.select(pl.col("Időpont").alias("Időpont_")),
        PV.select(pl.col("Időpont")),
    ],
    how="horizontal",
)

# for year in range(2019, 2025):
#    # print(PV.filter(pl.col("Naperőművek nettó üzemirányítási").is_null()))
#    print(
#        PV.filter(
#            (pl.col("Időpont").dt.year() == year)
#            & (pl.col("Időpont").dt.month() == 12)
#            & (pl.col("Időpont").dt.day() == 31)
#            & (pl.col("Időpont").dt.hour() == 23)
#            & (pl.col("Időpont").dt.minute() == 15)
#        )
#    )

print(df.filter(pl.col("Időpont") == pl.col("Időpont_")).tail())
print(df.filter(pl.col("Időpont") != pl.col("Időpont_")).head())

(210432, 14)
(210432, 13)
(210432, 6)
(210438, 15)
(210438, 5)
(210438, 11)
(210438, 8)

2019-01-01 01:00:00+01:00
2019-01-01 00:00:00+00:00
2018-12-31 23:15:00+00:00
2018-12-31 23:15:00+00:00
2018-12-31 23:15:00+00:00
2018-12-31 23:15:00+00:00
2018-12-31 23:15:00+00:00

2025-01-01 00:45:00+01:00
2024-12-31 23:45:00+00:00
2024-12-31 23:00:00+00:00
2024-12-31 23:15:00+00:00
2024-12-31 23:15:00+00:00
2024-12-31 23:15:00+00:00
2024-12-31 23:15:00+00:00

mavir_neg Időpont type: <class 'datetime.datetime'>
mavir_poz Időpont type: <class 'datetime.datetime'>
PV Időpont type: <class 'datetime.datetime'>
hatar_aramlas Időpont type: <class 'datetime.datetime'>
real_time_aggregated Időpont type: <class 'datetime.datetime'>
rendszerterheles Időpont type: <class 'datetime.datetime'>
wind Időpont type: <class 'datetime.datetime'>

mavir_neg Időpont sample:
shape: (5, 1)
┌───────────────────────────────┐
│ Időpont                       │
│ ---                           │
│ datetime[μs, Europe/Budape

SchemaError: could not evaluate comparison between series 'Időpont' of dtype: datetime[μs, UTC] and series 'Időpont_' of dtype: datetime[μs, Europe/Budapest]